In [1]:
%load_ext lab_black

In [2]:
# Loading all necessary libraries
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn import preprocessing
from dateutil import parser
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import spacy
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
import random
import unidecode
import emoji
import json
from pprint import pprint
import gc
from datetime import datetime
from datetime import timedelta
import os
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
from dateutil import parser
from datetime import timezone
import pytz
from sklearn.decomposition import LatentDirichletAllocation as LDA

In [3]:
%matplotlib inline

# sp = spacy.load("it_core_news_lg")

### Load Input Data

In [4]:
TYPE_PAGE_LIST = ["politicians"]
TYPE_SOCIAL_LIST = ["facebook"]  # ["facebook", "instagram"]
COUNTRY_PAGE_LIST = ["italy"]

DATA_PATH = "data/"
OUTPUT_PATH = "output/"

COLUMNS_TYPES_FB = {}
COLUMNS_TYPES_IG = {}

In [5]:
MONTHS_2019 = {"dec": 12}
MONTHS_2020 = {
    "jan": 1,
    "feb": 2,
    "mar": 3,
    "apr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "aug": 8,
}
df_map = {}
df_months_map = {
    "dec": {},
    "jan": {},
    "feb": {},
    "mar": {},
    "apr": {},
    "may": {},
    "jun": {},
    "jul": {},
    "aug": {},
}
df_specific_party_map = {
    "dec": {},
    "jan": {},
    "feb": {},
    "mar": {},
    "apr": {},
    "may": {},
    "jun": {},
    "jul": {},
    "aug": {},
}
df_pol_party_frequency_migration_map = {
    "dec": {},
    "jan": {},
    "feb": {},
    "mar": {},
    "apr": {},
    "may": {},
    "jun": {},
    "jul": {},
    "aug": {},
}
df_pol_party_frequency_risk_map = {
    "dec": {},
    "jan": {},
    "feb": {},
    "mar": {},
    "apr": {},
    "may": {},
    "jun": {},
    "jul": {},
    "aug": {},
}
df_pol_party_top_risk_map = {
    "dec": {},
    "jan": {},
    "feb": {},
    "mar": {},
    "apr": {},
    "may": {},
    "jun": {},
    "jul": {},
    "aug": {},
}
lists_months_map = {
    "dec": {},
    "jan": {},
    "feb": {},
    "mar": {},
    "apr": {},
    "may": {},
    "jun": {},
    "jul": {},
    "aug": {},
}
top_posts_map = {
    "dec": {},
    "jan": {},
    "feb": {},
    "mar": {},
    "apr": {},
    "may": {},
    "jun": {},
    "jul": {},
    "aug": {},
}
risk_lists_months_map = {
    "dec": {},
    "jan": {},
    "feb": {},
    "mar": {},
    "apr": {},
    "may": {},
    "jun": {},
    "jul": {},
    "aug": {},
}
risk_top_lists_months_map = {
    "dec": {},
    "jan": {},
    "feb": {},
    "mar": {},
    "apr": {},
    "may": {},
    "jun": {},
    "jul": {},
    "aug": {},
}


post_id_range_map = {}
account_df_map = {}
account_df_delta_map_final = {}
account_rolling_map = {}

immigration_words_list = [
    "immigrazione",
    "migranti",
    "migrante",
    "immigrato",
    "immigrati",
    "decreti sicurezza",
    "decreto sicurezza",
    "regolarizzazione",
    "sanatoria",
    "ong",
    "clandestini",
    "clandestino",
    "barconi",
    "barcone",
    "barchini",
    "lampedusa",
    "scafisti",
    "scafista",
    "profughi",
    "profugo",
    "carola rackete",
    "ius soli",
    "naufragio",
    "naufragi",
    "diritto di asilo",
    "guardia costiera",
    "trafficante",
    "trafficanti",
    "sbarcare",
    "sbarco",
    "sbarchi",
    "nomadi",
    "porti",
    "permessi di soggiorno",
    "centri di accoglienza",
    "centro di accoglienza",
    "blocco navale",
    "alan kurdi",
    "open arms",
    "gregoretti",
    "ocean viking",
    "sea watch",
    "moby zaza",
    "augusta",
    "mare jonio",
    "cpr",
    "porto empedocle",
    "rotta balcanica",
    "ventimiglia",
    "mondragone",
    "don biancalani",
    "pozzallo",
    "musumeci",
    "celio",
]

In [6]:
red_words_list = [
    "africani",
    "africa",
    "africano",
    "islamici",
    "tunisini",
    "libici",
    "tunisia",
    "libia",
    "musulmani",
    "irregolari",
    "ricollocamenti",
    "respingimenti",
    "mimmo lucano",
    "treviso",
]

In [7]:
with open(DATA_PATH + "risk_keywords_month.json") as json_file:
    risk_words_list = json.load(json_file)["risk_words"]

account_risk_ts_map = {}
account_red_ts_map = {}

In [8]:
immigration_words_list = [elem.strip().lower() for elem in immigration_words_list]

In [9]:
dateparse = lambda x: parser.parse(x, tzinfos={"EDT": "UTC-4", "EST": "UTC-5"})
df_time_series = pd.read_csv(
    "data/politicians_facebook_italy.csv",
    header=0,
    parse_dates=["Created"],
    date_parser=dateparse,
)

/Users/silvio/opt/anaconda3/envs/ricerca/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
def fix_timezone(date_elem):
    return date_elem.astimezone(pytz.utc)

In [11]:
df_time_series["Created"] = df_time_series["Created"].apply(lambda x: fix_timezone(x))

In [12]:
df_time_series.head()

,Page Name,User Name,Facebook Id,Likes at Posting,Created,Type,Likes,Comments,Shares,Love,...,URL,Message,Link,Final Link,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Overperforming Score
0,Lega - Salvini Premier,legasalvinipremier,422703967772535,764369,2020-09-01 13:36:22+00:00,Photo,3,0,1,0,...,https://www.facebook.com/legasalvinipremier/po...,"IL MESSAGGIO A SALVINI DI PIA KLEMP, LA CAPITA...",https://www.facebook.com/legasalvinipremier/ph...,NaN,@LegaSalviniPremier Süddeutsche Zeitung II tuo...,NaN,NaN,NaN,NaN,-17.25
1,MoVimento 5 Stelle,movimentocinquestelle,174457180812,1478039,2020-09-01 13:33:59+00:00,Link,0,0,0,0,...,https://www.facebook.com/movimentocinquestelle...,Oggi alla sorella di Alessandro Cannavacciuolo...,https://www.ilblogdellestelle.it/2020/09/la-so...,NaN,NaN,La solidarietà del MoVimento ad Alessandro Can...,Alessandro Cannavacciuolo è un ragazzo che da ...,NaN,NaN,-60
2,Lega - Salvini Premier,legasalvinipremier,422703967772535,764369,2020-09-01 13:31:01+00:00,Photo,46,1,2,2,...,https://www.facebook.com/legasalvinipremier/po...,"MENO 19 GIORNI! #IOVOTOLEGA Anche in Puglia, d...",https://www.facebook.com/legasalvinipremier/ph...,NaN,DOMENICA 20SETTEMBRE DALLE 7 ALLE 23 LUNEDÌ 21...,Timeline Photos,NaN,NaN,NaN,-1.33
3,Lega - Salvini Premier,legasalvinipremier,422703967772535,764369,2020-09-01 13:28:55+00:00,Photo,54,26,53,1,...,https://www.facebook.com/legasalvinipremier/po...,217 IMMIGRATI CLANDESTINI PORTATI DA LAMPEDUSA...,https://www.facebook.com/legasalvinipremier/ph...,NaN,NaN,NaN,NaN,NaN,NaN,3.26
4,Lega - Salvini Premier,legasalvinipremier,422703967772535,764369,2020-09-01 13:21:53+00:00,Link,15,11,5,0,...,https://www.facebook.com/legasalvinipremier/po...,LE ACCUSE DEL LEADER ISLAMICO: CRISTIANI ED EB...,https://www.ilgiornale.it/news/cronache/segret...,NaN,NaN,Le accuse del leader islamico: cristiani ed eb...,Le parole sconcertanti pubblicate su Facebook ...,NaN,NaN,-3.22


In [13]:
df_time_series.dtypes

Page Name                                      object
User Name                                      object
Facebook Id                                     int64
Likes at Posting                                int64
Created                           datetime64[ns, UTC]
Type                                           object
Likes                                           int64
Comments                                        int64
Shares                                          int64
Love                                            int64
Wow                                             int64
Haha                                            int64
Sad                                             int64
Angry                                           int64
Care                                            int64
Video Share Status                             object
Post Views                                      int64
Total Views                                     int64
Total Views For All Crosspos

In [14]:
df_time_series[["Message"]] = df_time_series[["Message"]].fillna(value="")

In [15]:
df_time_series["Message"] = df_time_series["Message"].str.lower()

In [16]:
def clean_date(row):
    s_date = str(row["Created"])
    new_date = s_date.split(" ")[0]
    new_date = datetime.strptime(new_date, "%Y-%m-%d")
    return new_date

In [17]:
df_time_series["date_clean"] = df_time_series.apply(lambda row: clean_date(row), axis=1)

#### First filter for immigration words

In [18]:
df_time_series = df_time_series[
    df_time_series["Message"].str.contains("|".join(immigration_words_list))
]

In [19]:
len(df_time_series)

6516

In [20]:
from collections import defaultdict

In [21]:
account_other_ts_map = {}
groups_by_account_other = df_time_series.groupby(["Page Name"])
for account_name, df_account in groups_by_account_other:
    list_of_lists_other = []
    groups_date = df_account.groupby("date_clean")
    # print(account_name)
    for date_, df_account_date in groups_date:
        # print(date_, len(df_account_date))
        list_of_lists_other.append([date_, len(df_account_date)])
    new_df = pd.DataFrame(list_of_lists_other, columns=["date", "n_posts"])
    new_df = new_df.set_index("date")
    new_df.index.names = ["date"]
    new_df = new_df.sort_index()
    account_other_ts_map[account_name] = new_df

In [22]:
idxs = pd.date_range("12-01-2019", "09-01-2020")

In [23]:
for name, df_ in account_other_ts_map.items():
    df_ = df_.reindex(idxs, fill_value=0)
    account_other_ts_map[name] = df_

#### Second filter for red words

In [24]:
df_time_series_red_words = df_time_series[
    df_time_series["Message"].str.contains("|".join(red_words_list))
].copy()

In [25]:
len(df_time_series_red_words)

301

In [26]:
groups_by_account_red = df_time_series_red_words.groupby(["Page Name"])
for account_name, df_account in groups_by_account_red:
    list_of_lists_red = []
    groups_date = df_account.groupby("date_clean")
    for date_, df_account_date in groups_date:
        list_of_lists_red.append([date_, len(df_account_date)])
    new_df = pd.DataFrame(list_of_lists_red, columns=["date", "n_posts"])
    new_df = new_df.set_index("date")
    new_df.index.names = ["date"]
    new_df = new_df.sort_index()
    account_red_ts_map[account_name] = new_df

In [27]:
for name, df_ in account_red_ts_map.items():
    df_ = df_.reindex(idxs, fill_value=0)
    account_red_ts_map[name] = df_

#### Third filter for risk words

In [28]:
df_time_series_risk_words = df_time_series[
    df_time_series["Message"].str.contains("|".join(risk_words_list))
].copy()

In [29]:
len(df_time_series_risk_words)

2475

In [30]:
groups_by_account_risk = df_time_series_risk_words.groupby(["Page Name"])
for account_name, df_account in groups_by_account_risk:
    list_of_lists_risk = []
    groups_date = df_account.groupby("date_clean")
    for date_, df_account_date in groups_date:
        list_of_lists_risk.append([date_, len(df_account_date)])
    new_df = pd.DataFrame(list_of_lists_risk, columns=["date", "n_posts"])
    new_df = new_df.set_index("date")
    new_df.index.names = ["date"]
    new_df = new_df.sort_index()
    account_risk_ts_map[account_name] = new_df

In [31]:
for name, df_ in account_risk_ts_map.items():
    df_ = df_.reindex(idxs, fill_value=0)
    account_risk_ts_map[name] = df_

In [32]:
parties = [
    "Fratelli d'Italia",
    "Lega - Salvini Premier",
    "MoVimento 5 Stelle",
    "Partito Democratico",
]
politicians = [
    "Giorgia Meloni",
    "Giuseppe Conte",
    "Luigi Di Maio",
    "Matteo Salvini",
    "Nicola Fratoianni",
]

In [33]:
account_risk_ts_map.keys()

dict_keys(['Emma Bonino', 'Forza Italia', "Fratelli d'Italia", 'Giorgia Meloni', 'Giuseppe Conte', 'Italia Viva', 'Lega - Salvini Premier', 'Luigi Di Maio', 'Matteo Renzi', 'Matteo Salvini', 'MoVimento 5 Stelle', 'Nicola Fratoianni', 'Nicola Zingaretti', 'Partito Democratico', 'Più Europa', 'Silvio Berlusconi'])

### Plots

In [34]:
def plot_time_series(acc_list, map_, title, file_name):
    plt.figure(figsize=(15, 10))
    plt.grid(True)
    for acc in [acc_list]:
        if acc in map_:
            plt.plot(
                map_[acc]["n_posts"], label=acc,
            )
        else:
            plt.close()
            return
    plt.axvspan(
        datetime(2020, 2, 21),
        datetime(2020, 5, 4),
        ymin=0,
        ymax=6,
        alpha=0.2,
        color="red",
    )
    # plt.linewidth = 5
    plt.legend(loc=2)
    plt.title("n. posts - {} keywords ".format(title))
    plt.savefig("output/timeseries/" + file_name + ".pdf")
    plt.close()

In [35]:
list_account_final = [
    "Matteo Salvini",
    "Giorgia Meloni",
    "MoVimento 5 Stelle",
    "Partito Democratico",
    "Giuseppe Conte",
]

In [36]:
for acc in list_account_final:
    plot_time_series(acc, account_risk_ts_map, "risk", "risk/{}".format(acc))

In [37]:
for acc in list_account_final:
    plot_time_series(acc, account_red_ts_map, "red", "red_words/{}".format(acc))

In [38]:
gc.collect()

31197

In [39]:
account_red_ts_map.keys()

dict_keys(['Emma Bonino', 'Forza Italia', "Fratelli d'Italia", 'Giorgia Meloni', 'Italia Viva', 'Lega - Salvini Premier', 'Luigi Di Maio', 'Matteo Salvini', 'MoVimento 5 Stelle', 'Nicola Fratoianni', 'Nicola Zingaretti', 'Partito Democratico', 'Più Europa', 'Silvio Berlusconi'])

In [40]:
account_risk_ts_map["Giorgia Meloni"]

,n_posts
2019-12-01,1
2019-12-02,0
2019-12-03,0
2019-12-04,0
2019-12-05,0
...,...
2020-08-28,1
2020-08-29,2
2020-08-30,2
2020-08-31,0


### Plot with frequency of risk keywords posts and other posts

In [41]:
def plot_time_series_comparation(acc_list, title, file_name):
    plt.figure(figsize=(15, 10))
    plt.grid(True)
    for acc in [acc_list]:
        if acc in account_other_ts_map and acc in account_risk_ts_map:
            plt.plot(
                account_other_ts_map[acc]["n_posts"],
                label=acc + " - All Immigration Posts",
            )
            plt.plot(
                account_risk_ts_map[acc]["n_posts"],
                label=acc + " - Only Risk and Threat Posts",
            )
        else:
            plt.close()
            return
    plt.axvspan(
        datetime(2020, 2, 21),
        datetime(2020, 5, 4),
        ymin=0,
        ymax=6,
        alpha=0.2,
        color="red",
    )
    # plt.linewidth = 5
    plt.legend(loc=2)
    plt.title("n.posts vs n.risk and threat posts, per day".format(title))
    plt.savefig("output/timeseries/" + file_name + ".pdf")
    plt.close()

In [42]:
for acc in list_account_final:
    plot_time_series_comparation(acc, "risk", "risk_compare_all_posts/{}".format(acc))

### Top 10 posts

In [45]:
def get_best_posts(df):
    df["interactions"] = (
        df["Likes"]
        + df["Comments"]
        + df["Shares"]
        + df["Love"]
        + df["Haha"]
        + df["Wow"]
        + df["Sad"]
        + df["Care"]
        + df["Angry"]
    )
    df = df.sort_values(
        by=["interactions", "Overperforming Score"], ascending=False
    ).iloc[0:10]
    return df

#### Top 10 posts - immigration posts

In [46]:
df_fb_list = []
grouped = df_time_series.groupby(["Page Name"])
for name, group in grouped:
    df_copy = group.copy()
    df = get_best_posts(df_copy)
    df_fb_list.append(df)
    # print(df.head())

In [47]:
result_fb = pd.concat(df_fb_list).reset_index()
result_fb.to_csv("output/top_10_posts_fb.csv")

#### Top 10 posts - immigration and risk and threat posts

In [48]:
df_fb_list = []
grouped = df_time_series_risk_words.groupby(["Page Name"])
for name, group in grouped:
    df_copy = group.copy()
    df = get_best_posts(df_copy)
    df_fb_list.append(df)
    # print(df.head())

In [49]:
result_fb = pd.concat(df_fb_list).reset_index()
result_fb.to_csv("output/top_10_posts_risk.csv")